# Data management using sqlite and pandas package

#### Here are some simple sqlite queries

In [1]:
import sqlite3
import pandas as pd

First, I create table with data

In [2]:
conn = sqlite3.connect('dataset_products') 
c = conn.cursor()

c.execute('''
          CREATE TABLE IF NOT EXISTS products_table
          ([product_id] INTEGER PRIMARY KEY, [product_name] TEXT, [product_price] INTEGER)
          ''')
          
c.execute('''
          INSERT INTO products_table (product_id, product_name, product_price)

                VALUES
                (1,'laptop',1700),
                (2,'tv',1300),
                (3,'fridge',800),
                (4,'laptop',1450),
                (5,'washing machine',650),
                (6,'bike',600),
                (7,'pc',3200),
                (8,'dryer',50),
                (9,'laptop',1950),
                (10,'washing machine',420),
                (11,'fridge',300),
                (12,'tv',1700),
                (13,'dryer',30),
                (14,'laptop',3500),
                (15,'microwave',150),
                (16,'bike',280),
                (17,'pc',2300),
                (18,'microwave',100),
                (19,'laptop',1550),
                (20,'dryer',100)
          ''')                     

conn.commit()


After creating the data, I check if everything is correct using the simplest SQL query

In [3]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM products_table
                               ''', conn)

df = pd.DataFrame(sql_query, columns = ['product_id', 'product_name', 'product_price'])
df.head(20)

,product_id,product_name,product_price
0,1,laptop,1700
1,2,tv,1300
2,3,fridge,800
3,4,laptop,1450
4,5,washing machine,650
5,6,bike,600
6,7,pc,3200
7,8,dryer,50
8,9,laptop,1950
9,10,washing machine,420


When everything is fine, I can start creating specific queries.

As a warm-up, I will check products with a price whose value exceeds 1000 and sort them

In [4]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               product_name, product_price
                               FROM products_table
                               WHERE product_price > 1000
                               ORDER BY product_price
                               ''', conn)

df = pd.DataFrame(sql_query, columns = ['product_name', 'product_price'])
df.head(10)

,product_name,product_price
0,tv,1300
1,laptop,1450
2,laptop,1550
3,laptop,1700
4,tv,1700
5,laptop,1950
6,pc,2300
7,pc,3200
8,laptop,3500


The next step will be to check the average price for a given type of product

In [5]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               product_name, AVG(product_price) AS avg_price
                               FROM products_table
                               GROUP BY product_name  
                               ''', conn)

df = pd.DataFrame(sql_query, columns = ['product_name', 'avg_price'])
df.head(10)

,product_name,avg_price
0,bike,440.0
1,dryer,60.0
2,fridge,550.0
3,laptop,2030.0
4,microwave,125.0
5,pc,2750.0
6,tv,1500.0
7,washing machine,535.0


The code below counts how many products of a given type we have

In [6]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               product_name, COUNT(product_name) as amount 
                               FROM products_table
                               GROUP BY product_name  
                               ''', conn)

df = pd.DataFrame(sql_query, columns = ['product_name', 'amount'])
df.head(20)

,product_name,amount
0,bike,2
1,dryer,3
2,fridge,2
3,laptop,5
4,microwave,2
5,pc,2
6,tv,2
7,washing machine,2


Finally, a slightly more advanced query that is used to categorize products depending on their purpose

In [7]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               CASE WHEN product_name = "fridge" OR product_name = "washing machine"  THEN "BIG AGD" 
                                    WHEN product_name = "microwave" OR product_name = "dryer"  THEN "SMALL AGD"
                                    WHEN product_name = "tv" OR product_name = "pc" OR product_name = "laptop" THEN "RTV"    
                               ELSE "other products" END AS product_type,
                               product_name, product_price
                               FROM products_table    
                               ORDER BY product_price DESC
                               ''', conn)

df = pd.DataFrame(sql_query, columns = ['product_name', 'product_price', "product_type"])
df.head(20)

,product_name,product_price,product_type
0,laptop,3500,RTV
1,pc,3200,RTV
2,pc,2300,RTV
3,laptop,1950,RTV
4,laptop,1700,RTV
5,tv,1700,RTV
6,laptop,1550,RTV
7,laptop,1450,RTV
8,tv,1300,RTV
9,fridge,800,BIG AGD
